In [2]:
# Dependencies
import numpy as np
import pandas as pd

In [3]:
censuses_data = pd.read_csv("./CSV/censuses.csv")
censuses_data = censuses_data[censuses_data["census_year"] == 1900]
censuses_data.head()

,line_number,house_number,street_name,last_name,given_name,relation,color_race,sex,age_at_last_birthday,occupation,own_or_rent,owned_free_or_mortgage,house_or_farm,my_notes,file_name,census_year
0,65,830,Arthur St,Brown,Eliza,Boarder,B,F,26,Servant,Null,Null,Null,Null,1900 census ED 70 page 2,1900
1,63,830,Arthur St,Butcher,John H.,Boarder,B,M,27,Drayman,Null,Null,Null,Null,1900 census ED 70 page 2,1900
2,61,830,Arthur St,Olliver,Marsha,Head,B,F,25,Laundress,Rent,Null,House,Null,1900 census ED 70 page 2,1900
3,64,830,Arthur St,Spivey,Neata,Boarder,B,F,25,Servant,Null,Null,Null,Null,1900 census ED 70 page 2,1900
4,62,830,Arthur St,Wells,Ethel,Daughter,B,F,8,At School,Null,Null,Null,Null,1900 census ED 70 page 2,1900


In [4]:
# Separate out the occupation which are null/ none (LATER we can test our model again these cases)
occupation_missing_mask = (censuses_data["occupation"].str.contains('None')) | (censuses_data["occupation"].str.contains('Null'))
df_missing_occupation = censuses_data[occupation_missing_mask]

class_occ = 0 #for missing occupation create new column named class_occ & set its value to 0
df_missing_occupation.loc[:,"class_occ"] = class_occ

df_missing_occupation.head()

D:\Users\kshah\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Users\kshah\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,line_number,house_number,street_name,last_name,given_name,relation,color_race,sex,age_at_last_birthday,occupation,own_or_rent,owned_free_or_mortgage,house_or_farm,my_notes,file_name,census_year,class_occ
6,67,832,Arthur St,Davis,Maria,Wife,B,F,34,Null,Null,Null,Null,Null,1900 census ED 70 page 2,1900,0
8,69,832,Arthur St,Davis,Della,Daughter,B,F,15,Null,Null,Null,Null,Null,1900 census ED 70 page 2,1900,0
10,71,832,Arthur St,Davis,Laurine,Son,B,M,9,Null,Null,Null,Null,Null,1900 census ED 70 page 2,1900,0
11,72,832,Arthur St,Davis,Arinda,Daughter,B,F,8,Null,Null,Null,Null,Null,1900 census ED 70 page 2,1900,0
13,79,834,Arthur St,Baily,Walter,Grandson,B,M,5,Null,Null,Null,Null,Null,1900 census ED 70 page 2,1900,0


In [5]:
# Prepare our X & y to create model with labelled 'occupation' column

# prepare new df where occupation column is All labelled
df_all_occupation = censuses_data[occupation_missing_mask == False]

class_occ = 1 # for labelled occupation create new column named class_occ & set its value to 1
df_all_occupation.loc[:,"class_occ"] = class_occ
# frames = [df_missing_occupation,df_all_occupation]
merge_data = pd.concat([df_missing_occupation,df_all_occupation])

# include following features that MAY influence occupation field -
# color_race, sex, age_at_last_birthday, occupation
newdata = merge_data.loc[:,["color_race","sex","age_at_last_birthday", "class_occ"]]
newdata.head()
# newdata["color_race"].replace("B", 1, inplace=True) # Replace B by 1
# newdata["color_race"].replace("W", 0, inplace=True) # Replace W by 0

# newdata["sex"].replace("M", 1, inplace=True) # Replace M by 1
# newdata["sex"].replace("F", 0, inplace=True) # Replace F by 0


,color_race,sex,age_at_last_birthday,class_occ
6,B,F,34,0
8,B,F,15,0
10,B,M,9,0
11,B,F,8,0
13,B,M,5,0


In [6]:
# # Create dictionary to convert occupation into unique numbers
# occupation = {}
# counter = 1
# for occ in newdata["occupation"].unique():
#     occupation[occ] = counter
#     counter += 1
# print(occupation)

# newdata["occupation"].replace(occupation, inplace=True) # replace occupation text by number
# newdata.head()

In [7]:
def age_clean(data,under_1_value = 1, pandas = 'yes'):
    clean_age = []
    if pandas == 'yes':
        data_lst = data
    else:
        data_lst = data
    for datum in data_lst:
        if 'month' in str(datum):
            if 'and' in str(datum):
                clean_age.append(str(datum).split(' ')[0])
            else:
                clean_age.append(str(under_1_value))
        else:
            clean_age.append(datum)
    return clean_age

In [8]:
age = newdata["age_at_last_birthday"]
clean_age = age_clean(newdata['age_at_last_birthday'])
print(type(clean_age))
newdata["age_at_last_birthday"] = clean_age
newdata.count()

<class 'list'>


color_race              808
sex                     808
age_at_last_birthday    808
class_occ               808
dtype: int64

In [9]:
X = newdata.drop("class_occ", axis=1)
y = newdata["class_occ"]

print(X.shape, y.shape)

(808, 3) (808,)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=1/2)
print(X_train.shape, y_train.shape)

(404, 3) (404,)


In [11]:
X_test.count()

color_race              404
sex                     404
age_at_last_birthday    404
dtype: int64

In [12]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_ohe = ohe.fit_transform(X_train)
X_test_ohe = ohe.transform(X_test)

print(X_train_ohe.shape, X_test_ohe.shape)

(404, 75) (404, 75)


In [13]:
from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression()

# classifier.fit(X_train_ohe,y_train)

clf = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial').fit(X_train_ohe,y_train)

print(f"Training Data Score: {clf.score(X_train_ohe, y_train)}")
print(f"Testing Data Score: {clf.score(X_test_ohe, y_test)}")

Training Data Score: 0.8465346534653465
Testing Data Score: 0.7623762376237624


In [20]:
for label, original_class in zip(X_train_ohe, X_train):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: color_race
Encoded Label: [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
------------
Original Class: sex
Encoded Label: [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
------------
Original Class: age_at_last_birthday
Encoded Label: [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
------------
